In [ ]:
# fortune 100 companies
# of 100 companies, 84 are public
# of 84 public, 74 start before 2004, 10 after
# cluster these 74 companies
# log diff correlation distance

rm(list=ls())

setwd('C:\\Users\\PLS2041\\Desktop\\PROJECT_GAMMA\\fortune_100_historical_data' )

##############################################################################

clean = function(df,title)
{
  tdf = subset(df,select=c('Date','Adj.Close'))
  
  names(tdf)[names(tdf)=='Adj.Close'] <- title
  
  adjCloseNum = as.numeric(tdf[,title])
  
  Log = log(adjCloseNum)
  
  diffLog = diff(Log)
  
  result = c(0.0,diffLog)

  tdf[,title] = result
  
  return(tdf)
}

full_name = function(stocks)
{
  fname = "C:\\Users\\PLS2041\\Desktop\\PROJECT_GAMMA\\__NAMES__.csv"
  
  names = read.csv(file=fname, header=TRUE, sep=",")
  names$full = as.character(names$full)
  names$abbrev = as.character(names$abbrev)
  
  for( i in 1:length(stocks) )
  {
    idx = match( stocks[i] , names$abbrev )
    
    stocks[i] = names$full[idx]
  }
  
  return(stocks)
}

kick_all = function(heapcsv,badapples)
{
  for(i in 1:length(badapples) )
  {
    heapcsv = kick_single_item(heapcsv,badapples[i])
  }
  
  return(heapcsv)
}

kick_single_item = function(heapcsv,badapple)
{
  preN = length(heapcsv)
  
  badapple = paste(badapple,'.csv',sep="")
  
  heapcsv = heapcsv[heapcsv != badapple]
  
  postN = length(heapcsv)
  
  stopifnot( postN < preN ) # check if removal occured
  
  return(heapcsv)
}

##############################################################################

library(stringr) # string manipulation

stocksCSV = list.files()

badlist = c('PSX','DVMT','FB','MPC','HCA','GM','CHTR','DAL','ETE','UAL')

stocksCSV = kick_all(stocksCSV,badlist)

stocks = str_sub(stocksCSV,1,str_length(stocksCSV)-4) # remove .csv

stocks = full_name(stocks)

n = length(stocks)

##############################################################################

for(i in 1:n)
{
  z = read.csv(file=stocksCSV[i], header=TRUE, sep=",")
  
  z = clean(z,stocks[i]) # RENAME HERE
  
  if( i==1 )
  {stockData = z}
  else
  {stockData = merge(stockData,z,by='Date')}
}

# 74 stocks + date column  = 75 variables

##############################################################################

stockData = na.omit(stockData)

stockData$Date = NULL

stockData = t(as.matrix(stockData))

##############################################################################

library(TSclust)

D = diss(stockData,"COR")

summary(D)

sort(rowMeans(as.matrix(D)))

tree = hclust(D)

plot(tree) # vanilla graphics

##############################################################################

library(factoextra)

library(ape)

require("igraph")

K = 32
# high level
fviz_dend(tree, k = K, cex = 1, horiz = TRUE,  k_colors = "jco",
          rect = TRUE, rect_border = "jco", rect_fill = TRUE)

H = 0.5
# high level
fviz_dend(tree, h = H, cex = 1, horiz = TRUE,
          rect = TRUE, rect_fill = TRUE)

phylo_setting = c("layout.auto", 
                  "layout_with_drl", 
                  "layout_as_tree", 
                  "layout.gem", 
                  "layout.mds", 
                  "layout_with_lgl")

# weird -> 4 good, 6 ok
fviz_dend(tree, 
          k = K, 
          cex = .85,
          k_colors = "jco",
          type = "phylogenic",            
          phylo_layout = phylo_setting[6],  
          color_labels_by_k = TRUE,
          ggtheme = theme_classic(),
          repel = TRUE)

library(dendextend)
library(colorspace)
k <- K
cols <- rainbow_hcl(k)
dend <- as.dendrogram(tree)
dend <- color_branches(dend, k = k)
plot(dend)
labels_dend <- labels(dend)
groups <- cutree(dend, k=K, order_clusters_as_data = FALSE)
dends <- list()
for(i in 1:k) {
  labels_to_keep <- labels_dend[i != groups]
  dends[[i]] <- prune(dend, labels_to_keep)
}

plot(dends[[17]])

